# Chapter 7: The CodeAct Pattern - Dynamic Computation

CodeAct lets agents **write and execute Python code on the fly**. No pre-built tools needed -- the agent generates code to solve problems dynamically.

When combined with tools, you get the best of both worlds: **tools for data fetching**, **code for analysis**.

In Chapter 5, we gave the agent tools. In Chapter 6, we watched it reason through multi-step problems. But here's a question those tools can't answer:

> "Calculate the Sharpe ratio for a portfolio with these monthly returns: 5%, 3%, -2%, 4%, 1%, -1%, 3%, 2%"

We don't have a `calculate_sharpe_ratio` tool. We could build one, but what about tomorrow's question? What if someone asks for the **Sortino ratio**? Or **Value at Risk**? Or a **custom metric** you've never heard of?

**You can't pre-build tools for every possible calculation.** This is where CodeAct comes in.

In [1]:
!pip install smolagents yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [2]:
from smolagents import CodeAgent, tool
from smolagents import OpenAIServerModel
from smolagents.monitoring import LogLevel

In [3]:
import getpass
API_KEY = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [4]:
model = OpenAIServerModel("gpt-4o-mini", api_key=API_KEY)
print("Model initialized!")

Model initialized!


## CodeAct: No Tools Needed

The key difference from previous chapters: we're **not passing any tools**. The agent's only capability is writing and executing Python code.

`CodeAgent` can write and execute Python code without any pre-built tools. The `additional_authorized_imports` parameter controls which packages the agent can use -- it acts as a **security whitelist**. Only explicitly listed packages can be imported by the generated code.

Why is executing code better than having the LLM reason in language? Because language-based math is unreliable -- an LLM might get a Sharpe ratio calculation approximately right, or it might silently drop a step. Code either runs correctly or throws an error. There's no "close enough" in Python. `2 + 2` is always `4`, never "approximately 4."

In [5]:
# CodeAgent with NO pre-built tools--only code generation
agent = CodeAgent(
    tools=[],  # No tools!
    model=model,
    verbosity_level=LogLevel.INFO,
    additional_authorized_imports=["numpy", "pandas"]  # Allow these imports
)

print("CodeAgent ready with NO tools -- only code generation!")

CodeAgent ready with NO tools -- only code generation!


## Example 1: Sharpe Ratio

The agent will write Python code to calculate the Sharpe ratio **from scratch**. No pre-built function needed.

Watch how the agent:
1. Thinks about what formula to use
2. Writes the Python code
3. Executes it and interprets the results

In [6]:
result = agent.run("""
Calculate the Sharpe ratio for a portfolio with these monthly returns:
5%, 3%, -2%, 4%, 1%, -1%, 3%, 2%

Assume a risk-free rate of 0.5% per month.
Show your calculation steps.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the Sharpe ratio for a portfolio with these monthly returns:                                          │
│ 5%, 3%, -2%, 4%, 1%, -1%, 3%, 2%                                                                                │
│                                                                                                                 │
│ Assume a risk-free rate of 0.5% per month.                                                                      │
│ Show your calculation steps.                                                                                    │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import numpy as np                                                                                               
                                                                                                                   
  # Given monthly returns and risk-free rate                                                                       
  returns = np.array([0.05, 0.03, -0.02, 0.04, 0.01, -0.01, 0.03, 0.02])                                           
  risk_free_rate = 0.005  # 0.5%                                                                                   
                                                                                                                   
  # Step 1: Calculate average return                                                                               
  average_return = np.mean(returns)                                                                                
  print(f"Average Portfolio Return: {average_return}")                                                             
                                                                                                                   
  # Step 2: Calculate excess return                                                                                
  excess_return = average_return - risk_free_rate                                                                  
  print(f"Excess Return: {excess_return}")                                                                         
                                                                                                                   
  # Step 3: Calculate standard deviation of returns                                                                
  std_dev = np.std(returns)                                                                                        
  print(f"Standard Deviation of Returns: {std_dev}")                                                               
                                                                                                                   
  # Step 4: Calculate Sharpe Ratio                                                                                 
  sharpe_ratio = excess_return / std_dev                                                                           
  print(f"Sharpe Ratio: {sharpe_ratio}")                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Average Portfolio Return: 0.018750000000000003
Excess Return: 0.013750000000000002
Standard Deviation of Returns: 0.022603926650031407
Sharpe Ratio: 0.6083013899702642

Out: None

[Step 1: Duration 8.28 seconds| Input tokens: 2,032 | Output tokens: 332]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(0.6083013899702642)                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 0.6083013899702642

[Step 2: Duration 3.33 seconds| Input tokens: 4,724 | Output tokens: 453]

## Example 2: Maximum Drawdown

Maximum drawdown measures the **largest peak-to-trough decline** in a portfolio's value. It answers the question: "What's the worst loss I could have experienced if I bought at the peak and sold at the bottom?"

The agent writes the code to compute this -- no pre-built tool required.

In [7]:
result = agent.run("""
Calculate the maximum drawdown for this sequence of portfolio values:
$100,000 -> $105,000 -> $98,000 -> $102,000 -> $95,000 -> $110,000

Explain what maximum drawdown means and show the calculation.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the maximum drawdown for this sequence of portfolio values:                                           │
│ $100,000 -> $105,000 -> $98,000 -> $102,000 -> $95,000 -> $110,000                                              │
│                                                                                                                 │
│ Explain what maximum drawdown means and show the calculation.                                                   │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  portfolio_values = [100000, 105000, 98000, 102000, 95000, 110000]                                                
                                                                                                                   
  peak = portfolio_values[0]                                                                                       
  max_drawdown = 0                                                                                                 
                                                                                                                   
  for value in portfolio_values:                                                                                   
      if value > peak:                                                                                             
          peak = value                                                                                             
      drawdown = peak - value                                                                                      
      if drawdown > max_drawdown:                                                                                  
          max_drawdown = drawdown                                                                                  
                                                                                                                   
  max_drawdown_percentage = (max_drawdown / peak) * 100  # Express as a percentage                                 
  max_drawdown_percentage                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: 9.090909090909092

[Step 1: Duration 5.81 seconds| Input tokens: 2,029 | Output tokens: 299]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(9.09)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 9.09

[Step 2: Duration 2.58 seconds| Input tokens: 4,535 | Output tokens: 406]

## Example 3: Correlation Analysis

The agent computes the **correlation coefficient** between two stocks' returns to assess diversification potential.

High positive correlation means stocks move together (poor diversification). Low or negative correlation means they move independently (good diversification).

In [8]:
result = agent.run("""
Given these monthly returns for two stocks:

Stock A: 2%, -1%, 3%, 1%, -2%, 4%
Stock B: 1%, 0%, 2%, 2%, -1%, 3%

Calculate the correlation coefficient between them.
Explain whether they would provide good diversification.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Given these monthly returns for two stocks:                                                                     │
│                                                                                                                 │
│ Stock A: 2%, -1%, 3%, 1%, -2%, 4%                                                                               │
│ Stock B: 1%, 0%, 2%, 2%, -1%, 3%                                                                                │
│                                                                                                                 │
│ Calculate the correlation coefficient between them.                                                             │
│ Explain whether they would provide good diversification.                                                        │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import numpy as np                                                                                               
                                                                                                                   
  # Monthly returns for Stock A and Stock B                                                                        
  returns_stock_A = np.array([0.02, -0.01, 0.03, 0.01, -0.02, 0.04])                                               
  returns_stock_B = np.array([0.01, 0.00, 0.02, 0.02, -0.01, 0.03])                                                
                                                                                                                   
  # Calculate the correlation coefficient                                                                          
  correlation_matrix = np.corrcoef(returns_stock_A, returns_stock_B)                                               
  correlation_coefficient = correlation_matrix[0, 1]                                                               
  print("Correlation Coefficient:", correlation_coefficient)                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Correlation Coefficient: 0.9286541181161709

Out: None

[Step 1: Duration 6.00 seconds| Input tokens: 2,041 | Output tokens: 309]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result_summary = {                                                                                               
      "correlation_coefficient": correlation_coefficient,                                                          
      "diversification_impact": "Not a good diversification due to high positive correlation."                     
  }                                                                                                                
  final_answer(result_summary)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'correlation_coefficient': np.float64(0.9286541181161709), 'diversification_impact': 'Not a good 
diversification due to high positive correlation.'}

[Step 2: Duration 4.36 seconds| Input tokens: 4,601 | Output tokens: 453]

## Combining Tools AND Code

Here's the really powerful thing: you can combine pre-built tools AND code generation.

The real power: use **tools to FETCH data** (from APIs), then let the agent **write code to ANALYZE it**. Tools handle I/O, code handles computation.

This is the pattern that scales: **tools for data, code for analysis**.

In [9]:
@tool
def get_historical_returns(ticker: str) -> list:
    """Get historical monthly returns for a stock (last 12 months).

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        List of monthly returns as decimals
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y", interval="1mo")
    returns = hist['Close'].pct_change().dropna().tolist()
    return returns

print("Tool created: get_historical_returns")

Tool created: get_historical_returns


In [10]:
agent_combined = CodeAgent(
    tools=[get_historical_returns],  # Tool for fetching data
    model=model,
    verbosity_level=LogLevel.INFO,
    additional_authorized_imports=["numpy", "pandas"]  # Code for analysis
)

print("Agent ready with tool + code generation!")

Agent ready with tool + code generation!


In [11]:
result = agent_combined.run("""
Compare the Sharpe ratios of AAPL, NVDA, and MSFT.
Use a risk-free rate of 0.3% per month.
Rank them from best to worst risk-adjusted return.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare the Sharpe ratios of AAPL, NVDA, and MSFT.                                                              │
│ Use a risk-free rate of 0.3% per month.                                                                         │
│ Rank them from best to worst risk-adjusted return.                                                              │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Getting historical returns for AAPL, NVDA, and MSFT                                                            
  aapl_returns = get_historical_returns("AAPL")                                                                    
  nvda_returns = get_historical_returns("NVDA")                                                                    
  msft_returns = get_historical_returns("MSFT")                                                                    
                                                                                                                   
  # Print the returns to check                                                                                     
  print("AAPL Returns:", aapl_returns)                                                                             
  print("NVDA Returns:", nvda_returns)                                                                             
  print("MSFT Returns:", msft_returns)                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
AAPL Returns: [-0.04335306057398958, -0.0548234432422523, 0.0228480764144936, 0.0116976749668114, 
0.11836967275862231, 0.09812603000704412, 0.061815166304167946, 0.031364424385052825, -0.024122217892287856, 
-0.045538079860853675, 0.017908595155657236]
NVDA Returns: [0.005074885758387104, 0.2406354284372363, 0.1691704107158205, 0.12590886882092356, 
-0.02074551060599117, 0.07119078244098076, 0.0853329169644188, -0.1258827854186917, 0.05367237434635719, 
0.024882783069967607, -0.0322293910152216]
MSFT Returns: [0.05293159369516842, 0.16470164597553816, 0.0824641742416028, 0.0725558001480362, 
-0.050253001762528005, 0.02390310838867782, -0.0002703583369538798, -0.049825164295963065, -0.015212838426091446, 
-0.11027250136616673, -0.07769184161725506]

Out: None

[Step 1: Duration 8.13 seconds| Input tokens: 2,072 | Output tokens: 264]

...

[Step 3: Duration 3.27 seconds| Input tokens: 8,515 | Output tokens: 751]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Ranked from best to worst Sharpe ratio: 1. NVDA, 2. AAPL, 3. MSFT")                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Ranked from best to worst Sharpe ratio: 1. NVDA, 2. AAPL, 3. MSFT

[Step 4: Duration 2.16 seconds| Input tokens: 12,471 | Output tokens: 849]

## Security: The Whitelist Approach

The `additional_authorized_imports` parameter acts as a **security whitelist**. Only explicitly allowed packages can be imported by the agent's generated code.

```python
agent = CodeAgent(
    tools=[],
    model=model,
    additional_authorized_imports=["numpy", "pandas"]  # ONLY these
)
```

This prevents the agent from importing dangerous modules like `os`, `subprocess`, or `requests`. The agent can't access the filesystem, run shell commands, or make network calls unless you explicitly allow it.

**In production**, you'd also want:
- **Sandboxed execution** -- code runs in an isolated environment
- **Resource limits** -- timeout after N seconds, limit memory usage
- **Logging** -- every code block is logged for audit
- **Review option** -- human approval before execution (for high-stakes scenarios)

## Exercise

Use the combined agent (tools + code) to analyze a portfolio. Fetch historical returns for 2-3 stocks and ask the agent to calculate a metric **not shown above**.

**Ideas:**
- "Calculate the **Sortino ratio** (like Sharpe but only penalizes downside volatility)"
- "Calculate the **tracking error** relative to a benchmark"
- "Calculate the **information ratio** (excess return over benchmark / tracking error)"
- "Calculate **Value at Risk (VaR)** at the 95% confidence level"

Include the data in your prompt, or let the tool fetch it for you. Watch the agent write and execute the code.

In [ ]:
result = agent_combined.run("""
    YOUR PROMPT HERE
""")